In [15]:
import pandas as pd

# Load the datasets
meta_fase_nota_aluno = pd.read_csv('csv_output/Tabelas/TbMeta/TbMetaFaseNotaAluno.csv')
meta_fase_nota = pd.read_csv('csv_output/Tabelas/TbMeta/TbMetaFaseNota.csv')
meta = pd.read_csv('csv_output/Tabelas/TbMeta/TbMeta.csv')
diario_frequencia = pd.read_csv('csv_output/Tabelas/TbDiario/Merge/TbDiarioFrequencia.csv')
diario_aula = pd.read_csv('csv_output/Tabelas/TbDiario/Originais anonimizados/TbDiarioAula.csv')

In [17]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [51]:
# Load the datasets
situacao_df = pd.read_csv('csv_output/Tabelas/TbSituacaoAlunoDisciplina/Originais anonimizados/TbSituacaoAlunoDisciplina.csv')
aluno_df = pd.read_csv('csv_output/Tabelas/TbAluno/Originais anonimizados/TbAluno.csv')

In [52]:
# Merge datasets on a common student ID
merged_df = pd.merge(situacao_df, aluno_df, on='IdAluno', how='inner')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21133 entries, 0 to 21132
Data columns (total 76 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   IdDisciplina                            21133 non-null  int64  
 1   IdTurma                                 21133 non-null  int64  
 2   IdAluno                                 21133 non-null  int64  
 3   IdFaseNotaAtual                         21133 non-null  int64  
 4   SituacaoAtual                           21133 non-null  object 
 5   StDisciplinaDispensada                  21133 non-null  object 
 6   IdAlunoCurso                            0 non-null      float64
 7   DataMatricula                           0 non-null      float64
 8   IdUsuarioMatricula                      17954 non-null  float64
 9   DataTrancamento                         0 non-null      float64
 10  IdUsuarioTrancamento                    0 non-null      fl

In [53]:
# Drop columns with no useful data
cols_to_drop = ['IdAlunoCurso', 'DataMatricula', 'DataTrancamento', 'IdMotivoTrancamento', 'NotaFinal', 
                'Faltas', 'NomeAbreviado', 'Naturalidade', 'Nacionalidade', 'TipoSangue', 'PlanoSaude', 
                'Alergia', 'Medicamento', 'IdEstabelecimentoEnsinoAnterior', 'IdReligiao', 'AnoConclusao', 
                'EnsinoMedio_IdEstabelecimentoEnsino', 'EnsinoMedio_AnoConclusao', 'NomeCivil', 'Unnamed: 55']

# Drop the columns
cleaned_df = merged_df.drop(columns=cols_to_drop)
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21133 entries, 0 to 21132
Data columns (total 56 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   IdDisciplina                            21133 non-null  int64  
 1   IdTurma                                 21133 non-null  int64  
 2   IdAluno                                 21133 non-null  int64  
 3   IdFaseNotaAtual                         21133 non-null  int64  
 4   SituacaoAtual                           21133 non-null  object 
 5   StDisciplinaDispensada                  21133 non-null  object 
 6   IdUsuarioMatricula                      17954 non-null  float64
 7   IdUsuarioTrancamento                    0 non-null      float64
 8   ComentarioTrancamento                   0 non-null      float64
 9   IdSituacaoAlunoDisciplina               21133 non-null  int64  
 10  ProblemaAutorizadoMatricula             0 non-null      fl

In [54]:
# Drop columns with more than 80% missing data
threshold = len(cleaned_df) * 0.8
cleaned_df = cleaned_df.dropna(thresh=threshold, axis=1)
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21133 entries, 0 to 21132
Data columns (total 45 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   IdDisciplina                            21133 non-null  int64  
 1   IdTurma                                 21133 non-null  int64  
 2   IdAluno                                 21133 non-null  int64  
 3   IdFaseNotaAtual                         21133 non-null  int64  
 4   SituacaoAtual                           21133 non-null  object 
 5   StDisciplinaDispensada                  21133 non-null  object 
 6   IdUsuarioMatricula                      17954 non-null  float64
 7   IdSituacaoAlunoDisciplina               21133 non-null  int64  
 8   StGoogleEdu_Sincronizado                21133 non-null  bool   
 9   NomeAluno                               21133 non-null  object 
 10  IdUnidade                               21123 non-null  fl

In [56]:
# Data Understanding
print(cleaned_df.describe())
print(cleaned_df.info())

       IdDisciplina       IdTurma       IdAluno  IdFaseNotaAtual  \
count  21133.000000  21133.000000  21133.000000     21133.000000   
mean       9.692282    412.699617    857.099134       442.849714   
std        8.005481    213.140641    570.060689       330.105841   
min        1.000000     11.000000      3.000000        15.000000   
25%        4.000000    200.000000    372.000000       224.000000   
50%        5.000000    454.000000    761.000000       330.000000   
75%       18.000000    615.000000   1234.000000       907.000000   
max       25.000000    760.000000   2241.000000      1087.000000   

       IdUsuarioMatricula  IdSituacaoAlunoDisciplina  IdUnidade   EstadoCivil  \
count        17954.000000               21133.000000    21123.0  21123.000000   
mean           109.408433               14652.728150        1.0      4.671448   
std            135.540591                8038.943985        0.0      1.238905   
min             13.000000                   9.000000        1.0

In [39]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21133 entries, 0 to 21132
Data columns (total 46 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   IdDisciplina                            21133 non-null  int64  
 1   IdTurma                                 21133 non-null  int64  
 2   IdAluno                                 21133 non-null  int64  
 3   IdFaseNotaAtual                         21133 non-null  int64  
 4   SituacaoAtual                           21133 non-null  object 
 5   StDisciplinaDispensada                  21133 non-null  object 
 6   IdUsuarioMatricula                      21133 non-null  float64
 7   IdSituacaoAlunoDisciplina               21133 non-null  int64  
 8   StGoogleEdu_Sincronizado                21133 non-null  bool   
 9   NomeAluno                               21133 non-null  object 
 10  IdUnidade                               21133 non-null  fl

In [59]:
cleaned_df

,IdDisciplina,IdTurma,IdAluno,IdFaseNotaAtual,SituacaoAtual,StDisciplinaDispensada,IdUsuarioMatricula,IdSituacaoAlunoDisciplina,StGoogleEdu_Sincronizado,NomeAluno,...,StRecursoTradutorInterpreteDeLibras,StRecursoLeituraLabial,StRecursoMaterialDidaticoProvaBraille,StRecursoProvaAmpliada,StRecursoProvaSuperampliada,StRecursoCdComAudio,StRecursoLinguaPortuguesaSegundaLingua,StRecursoProvaEmVideo,StPermiteUsoImagem,StEstrangeiro
0,1,106,3,55,A,A,15.0,9,False,Aluno 1,...,False,False,False,False,False,False,False,False,False,False
1,4,106,3,55,A,A,15.0,10,False,Aluno 1,...,False,False,False,False,False,False,False,False,False,False
2,5,106,3,55,A,A,15.0,11,False,Aluno 1,...,False,False,False,False,False,False,False,False,False,False
3,17,106,3,55,A,A,15.0,12,False,Aluno 1,...,False,False,False,False,False,False,False,False,False,False
4,5,113,5,67,A,A,16.0,13,False,Aluno 3,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21128,1,659,413,990,C,A,NaN,27458,False,Aluno 411,...,False,False,False,False,False,False,False,False,False,False
21129,4,659,413,993,C,A,NaN,27459,False,Aluno 411,...,False,False,False,False,False,False,False,False,False,False
21130,5,659,413,993,C,A,NaN,27460,False,Aluno 411,...,False,False,False,False,False,False,False,False,False,False
21131,22,659,413,991,C,A,NaN,27461,False,Aluno 411,...,False,False,False,False,False,False,False,False,False,False


In [72]:
cleaned_df['SituacaoAtual'].value_counts()

SituacaoAtual
C    19371
A     1762
Name: count, dtype: int64